# Module 4

In [1]:
!python --version

Python 3.12.12


In [2]:
!pip install transformers==4.57.1 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 25.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.4.1
    Uninstalling huggingface_hub-1.4.1:
      Successfully uninstalled huggingface_hub-1.4.1
  Attempting uninstall: transformers
    Found existing installation: transformers 5.2.0
    Uninstalling transformers-5.2.0:
      Successfully uninstalled transformers-5.2.0


In [3]:
!pip install trl==0.28.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 13.6 MB/s eta 0:00:00


In [4]:
!pip install peft==0.14.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.18.1
    Uninstalling peft-0.18.1:
      Successfully uninstalled peft-0.18.1


In [5]:
!pip install -U bitsandbytes>=0.46.1

In [6]:

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
wandb_token = user_secrets.get_secret("WANDB_API_KEY")

from huggingface_hub import login
login(token=hf_token)

import wandb
wandb.login(key=wandb_token)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24s004 (da24s004-iitm) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [7]:
import torch #
from datasets import load_dataset
from transformers import (
    AutoModelForImageTextToText,
    AutoProcessor, 
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer


2026-02-23 03:04:27.287722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771815867.616788      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771815867.714820      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771815868.529901      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771815868.529941      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771815868.529944      24 computation_placer.cc:177] computation placer alr

In [8]:
MODEL_ID = "google/medgemma-1.5-4b-it"
DATASET_PATH = "/kaggle/input/datasets/da24s004shrish/module4-data/module4_data"
OUTPUT_DIR = "/kaggle/working/module4_progression_lora"

NEW_DATA_PATH = "/kaggle/working/module4_data"

In [9]:
import shutil

shutil.copytree(
    DATASET_PATH,
    NEW_DATA_PATH
)

'/kaggle/working/module4_data'

In [10]:
print(f"Loading dataset from {NEW_DATA_PATH}...")
from datasets import load_from_disk

# Instantly loads the exact train and eval splits into memory
full_dataset = load_from_disk(NEW_DATA_PATH)

print("Loaded training samples:", len(full_dataset['train']))
print("Loaded eval samples:", len(full_dataset['eval']))
print("Loaded test samples:", len(full_dataset['test']))

Loading dataset from /kaggle/working/module4_data...
Loaded training samples: 531
Loaded eval samples: 76
Loaded test samples: 152


In [11]:
# --- 2. Processor & Chat Template Formatting ---
print("Initializing MedGemma 1.5 Processor...")
# MedGemma 1.5 uses a Processor instead of a standalone Tokenizer
processor = AutoProcessor.from_pretrained(MODEL_ID)

# We extract the underlying tokenizer to pass to the SFTTrainer for text processing
tokenizer = processor.tokenizer
tokenizer.padding_side = 'right' 
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Initializing MedGemma 1.5 Processor...


processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [12]:
# Function to apply MedGemma's specific multimodal chat template to our text array
def format_chat_template(example):
    example["text"] = processor.apply_chat_template(
        example["messages"], 
        tokenize=False, 
        add_generation_prompt=False
    )
    return example

In [13]:
train_dataset = full_dataset['train']
eval_dataset = full_dataset['eval']
test_dataset = full_dataset['test']

print("Formatting splits to MedGemma conversational structure...")
# Map the formatting function to both splits independently
train_dataset = train_dataset.map(format_chat_template)
eval_dataset = eval_dataset.map(format_chat_template)
test_dataset = test_dataset.map(format_chat_template)

Formatting splits to MedGemma conversational structure...


Map:   0%|          | 0/531 [00:00<?, ? examples/s]

Map:   0%|          | 0/76 [00:00<?, ? examples/s]

Map:   0%|          | 0/152 [00:00<?, ? examples/s]

In [14]:
# --- 3. 4-Bit Quantization Setup (QLoRA) ---
print("Loading Base Multimodal Model in 4-bit...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# CRITICAL: Load using the ImageTextToText class
model = AutoModelForImageTextToText.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

Loading Base Multimodal Model in 4-bit...


config.json:   0%|          | 0.00/2.55k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [15]:
# Prepare model for parameter-efficient fine-tuning
model = prepare_model_for_kbit_training(model)

In [16]:
# --- 4. LoRA Adapter Configuration ---
print("Injecting LoRA Adapters...")
peft_config = LoraConfig(
    r=16, # Rank: Controls the capacity of the adapter
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
new_model = get_peft_model(model, peft_config)
new_model.print_trainable_parameters()

Injecting LoRA Adapters...
trainable params: 32,788,480 || all params: 4,332,867,952 || trainable%: 0.7567


In [17]:
# --- 5. Training Arguments ---
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,      # Adjust based on your GPU VRAM
    per_device_eval_batch_size=1,   
    gradient_accumulation_steps=8,      # Simulates a batch size of 8
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    eval_strategy="steps",   
    logging_steps=20,
    eval_steps=20, 
    num_train_epochs=3,                 # 3 epochs is standard for medical instruction tuning
    max_steps=-1,
    fp16=False,
    bf16=True,                          # Use bf16 if you have an Ampere GPU (RTX 3000/4000/A100)
    report_to="wandb",                   
    run_name="medgemma_module4_1",
)

In [18]:

# --- 6. SFT Trainer Initialization ---
print("Initializing Supervised Fine-Tuning (SFT) Trainer...")
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset, 
    peft_config=peft_config,
    args=training_args,
)

Initializing Supervised Fine-Tuning (SFT) Trainer...


Tokenizing train dataset:   0%|          | 0/531 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/531 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/76 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/76 [00:00<?, ? examples/s]

In [19]:
# --- 7. Execute Training ---
print("Beginning training... This will take some time.")
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.


Beginning training... This will take some time.


wandb: Tracking run with wandb version 0.24.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20260223_030600-w0vxbqk0
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run medgemma_module4_1
wandb: ⭐️ View project at https://wandb.ai/da24s004-iitm/huggingface
wandb: 🚀 View run at https://wandb.ai/da24s004-iitm/huggingface/runs/w0vxbqk0
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
20,0.607100,0.462219
40,0.460700,0.447997
60,0.433900,0.419381
80,0.406000,0.402470
100,0.393600,0.390642
120,0.381900,0.387904
140,0.382500,0.385560
160,0.376600,0.384807
180,0.368600,0.383994
200,0.373500,0.383866


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=201, training_loss=0.4180573004098674, metrics={'train_runtime': 8160.4358, 'train_samples_per_second': 0.195, 'train_steps_per_second': 0.025, 'total_flos': 2.681715114527904e+16, 'train_loss': 0.4180573004098674})

In [20]:
# --- 8. Save the Final LoRA Adapter ---
print(f"Training complete. Saving adapter to {OUTPUT_DIR}...")
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ Module 2 Adapter successfully compiled and saved.")

Training complete. Saving adapter to /kaggle/working/module4_progression_lora...
✅ Module 2 Adapter successfully compiled and saved.


# Testing

In [21]:
import torch
import random
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import wandb

# 1. Put the model in evaluation mode
model.eval()

results = []

print(f"Starting inference on {len(test_dataset)} test samples...")

# 2. Iterate through the test set
for example in tqdm(test_dataset):
    # Extract just the user's prompt (ignore the assistant's target for input)
    user_msg = [msg for msg in example["messages"] if msg["role"] == "user"]
    true_target = [msg for msg in example["messages"] if msg["role"] == "assistant"][0]["content"]
    
    # Format the prompt exactly how MedGemma expects it for generation
    prompt = processor.apply_chat_template(
        user_msg, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    # Tokenize and move to GPU
    inputs = processor(text=prompt, return_tensors="pt", padding=True).to(model.device)
    
    # Generate the response
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=100, # Enough tokens to capture the "Yes/No" and reasoning
            do_sample=False,    # Greedy decoding for consistent diagnostic evaluation
            pad_token_id=processor.tokenizer.pad_token_id
        )
        
    # Decode only the newly generated tokens (ignore the input prompt)
    input_length = inputs["input_ids"].shape[1]
    generated_tokens = outputs[0, input_length:]
    pred_text = processor.decode(generated_tokens, skip_special_tokens=True).strip()
    
    # 3. Parse the binary classification (Extract "Yes" or "No")
    # Our Module 4 labels strictly start with "Yes." or "No."
    true_binary = "Yes" if true_target.startswith("Yes") else "No"
    pred_binary = "Yes" if pred_text.startswith("Yes") else "No" if pred_text.startswith("No") else "Invalid"
    
    is_correct = (true_binary == pred_binary)
    
    results.append({
        "User_Prompt": user_msg[0]["content"],
        "Ground_Truth_Text": true_target,
        "Model_Prediction_Text": pred_text,
        "True_Label": true_binary,
        "Predicted_Label": pred_binary,
        "Correct": is_correct
    })

print("Inference complete!")

Starting inference on 152 test samples...


100%|██████████| 152/152 [12:54<00:00,  5.09s/it]

Inference complete!


In [22]:
# Extract labels for scikit-learn
y_true = [r["True_Label"] for r in results]
y_pred = [r["Predicted_Label"] for r in results]

# Calculate Metrics (Treating "Yes" / Progression as the positive class)
accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(
    y_true, y_pred, labels=["Yes"], average="macro", zero_division=0
)

print("\n" + "="*40)
print("📊 TEST SET METRICS (Binary Progression)")
print("="*40)
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print("="*40 + "\n")

# Filter samples into categories
correct_samples = [r for r in results if r["Correct"]]
wrong_samples = [r for r in results if not r["Correct"]]

# Select 5 random samples
random_5 = random.sample(results, min(5, len(results)))

# Select top 3 correct and top 3 wrong (Since it's text generation, we just pick the first 3)
top_3_correct = correct_samples[:3]
top_3_wrong = wrong_samples[:3]

def print_samples(title, samples):
    print(f"\\n--- {title} ---")
    for i, s in enumerate(samples):
        print(f"\\nSample #{i+1}")
        print(f"  True Label: {s['True_Label']} | Predicted: {s['Predicted_Label']}")
        print(f"  Ground Truth: {s['Ground_Truth_Text']}")
        print(f"  Model Output: {s['Model_Prediction_Text']}")

print_samples("5 RANDOM SAMPLES", random_5)
print_samples("3 CORRECT PREDICTIONS", top_3_correct)
print_samples("3 WRONG PREDICTIONS", top_3_wrong)


📊 TEST SET METRICS (Binary Progression)
Accuracy:  0.9145
Precision: 0.7778
Recall:    0.6087
F1 Score:  0.6829

\n--- 5 RANDOM SAMPLES ---
\nSample #1
  True Label: No | Predicted: No
  Ground Truth: No immediate rapid progression detected. The M-Protein is currently stable. Continue routine longitudinal monitoring.
  Model Output: No immediate rapid progression detected. The M-Protein is currently stable. Continue routine longitudinal monitoring.
\nSample #2
  True Label: No | Predicted: No
  Ground Truth: No immediate rapid progression detected. The M-Protein is currently stable. Continue routine longitudinal monitoring.
  Model Output: No immediate rapid progression detected. The M-Protein is currently stable. Continue routine longitudinal monitoring.
\nSample #3
  True Label: Yes | Predicted: Yes
  Ground Truth: Yes. The velocity of the biomarkers indicates clinical disease progression according to IMWG criteria (>= 25% relative increase and >= 0.5 g/dL absolute increase in M-Pro

In [23]:
# Ensure wandb is still initialized from your training run
if wandb.run is None:
    print("Initializing new wandb run for evaluation...")
    wandb.init(project="huggingface", name="medgemma_module4_eval")

# 1. Log the numeric metrics
wandb.log({
    "test_accuracy": accuracy,
    "test_precision": precision,
    "test_recall": recall,
    "test_f1": f1
})

# 2. Build a WandB Table to log the text samples cleanly
columns = ["Category", "True Label", "Predicted Label", "Correct", "Ground Truth Text", "Model Output", "Input Prompt"]
test_table = wandb.Table(columns=columns)

# Helper function to add rows to the table
def add_to_table(category_name, samples_list):
    for s in samples_list:
        test_table.add_data(
            category_name, 
            s["True_Label"], 
            s["Predicted_Label"], 
            s["Correct"], 
            s["Ground_Truth_Text"], 
            s["Model_Prediction_Text"],
            s["User_Prompt"]
        )

# Add our categorized samples to the table
add_to_table("Random Sample", random_5)
add_to_table("Correct Prediction", top_3_correct)
add_to_table("Wrong Prediction", top_3_wrong)

# Log the table to your wandb dashboard
wandb.log({"Module_4_Test_Evaluation": test_table})

print("✅ Successfully logged all metrics and text samples to Weights & Biases!")

✅ Successfully logged all metrics and text samples to Weights & Biases!
